In [1]:
import requests

## 1 requests

In [2]:
url = 'https://www.pagina12.com.ar/'
p12 = requests.get(url)

In [3]:
p12.status_code
p12.text
p12.content
p12.headers
p12.request.headers
p12.request.method
p12.request.url

'https://www.pagina12.com.ar/'

## BS4

In [13]:
from bs4 import BeautifulSoup


In [15]:
s = BeautifulSoup(p12.text, 'html.parser')

In [16]:
type(s)

bs4.BeautifulSoup

In [ ]:
print(s.prettify())

In [ ]:
secciones = s.find('ul', attrs={'class':'main-sections'}).find_all('li')
secciones

In [25]:
[seccion.text for seccion in secciones]

['El país',
 'Economía',
 'Sociedad',
 'Cultura y Espectáculos',
 'Deportes',
 'El mundo',
 'Contratapa']

In [27]:
seccion = secciones[0]
seccion

<li class="p12-separator--right--primary"><a href="https://www.pagina12.com.ar/secciones/el-pais">El país</a></li>

In [28]:
seccion.a.get('href')


'https://www.pagina12.com.ar/secciones/el-pais'

In [29]:
links_secciones = [seccion.a.get('href') for seccion in secciones]
links_secciones

['https://www.pagina12.com.ar/secciones/el-pais',
 'https://www.pagina12.com.ar/secciones/economia',
 'https://www.pagina12.com.ar/secciones/sociedad',
 'https://www.pagina12.com.ar/suplementos/cultura-y-espectaculos',
 'https://www.pagina12.com.ar/secciones/deportes',
 'https://www.pagina12.com.ar/secciones/el-mundo',
 'https://www.pagina12.com.ar/secciones/contratapa']

In [30]:
sec = requests.get(links_secciones[0])

In [31]:
sec

<Response [200]>

In [32]:
sec.request.url

'https://www.pagina12.com.ar/secciones/el-pais'

In [33]:
soup_seccion = BeautifulSoup(sec.text, 'html.parser')

In [ ]:
print(soup_seccion.prettify())

In [35]:
featured_article = soup_seccion.find('div', attrs={'class':'featured-article__container'})
featured_article

In [ ]:
featured_article.a.get('href')

In [ ]:
article_list = soup_seccion.find('ul', attrs={'class':'article-list'})

In [ ]:
def obtener_notas(soup):
    '''
    Función que recibe un objeto de BeautifulSoup de una página de una sección
    y devuelve una lista de URLs a las notas de esa sección
    '''
    lista_notas = []
    
    # Obtengo el artículo promocionado
    featured_article = soup.find('div', attrs={'class':'featured-article__container'})
    if featured_article:
        lista_notas.append(featured_article.a.get('href'))
    
    # Obtengo el listado de artículos
    article_list = soup.find('ul', attrs={'class':'article-list'})
    for article in article_list.find_all('li'):
        if article.a:
            lista_notas.append(article.a.get('href'))
    
    return lista_notas

In [ ]:
lista_notas = obtener_notas(soup_seccion)
lista_notas

In [ ]:
r = requests.get(lista_notas[0])
if r.status_code == 200:
    # Procesamos la respuesta
    print('procesamos..')
else:
     # Informar el error
    print('informamos...')
url_nota = lista_notas[0]
print(url_nota)
url_mala = url_nota.replace('2','3')
print(url_mala)
try:
    nota = requests.get(url_mala)
except Exception as e:
    print('Error en la request:')
    print(e)
    print('\n')
    
print('El resto del programa continúa...')

In [ ]:
url_nota = lista_notas[0]
print(url_nota)

In [ ]:
try:
    nota = requests.get(url_nota)
    if nota.status_code == 200:
        s_nota = BeautifulSoup(nota.text, 'lxml')
        # Extraemos el título
        titulo = s_nota.find('div', attrs={'class':'article-title'})
        print(titulo.text)
        # Extraemos la fecha
        fecha = s_nota.find('span', attrs={'pubdate':'pubdate'}).get('datetime')
        print(fecha)
        # Extraemos la volanta
        volanta = s_nota.find('div', attrs={'class':'article-prefix'})
except Exception as e:
    print('Error:')
    print(e)
    print('\n')

In [ ]:
media = s_nota.find('div', attrs={'class':'article-main-media-image'})
imagenes = media.find_all('img')
imagenes

In [ ]:
if len(imagenes) == 0:
    print('no se encontraron imágenes')
else:
    imagen = imagenes[-1]
    img_src = imagen.get('data-src')
    print(img_src)
img_req = requests.get(img_src)

In [ ]:
from IPython.display import Image

In [ ]:
Image(img_req.content)